In [ ]:
!pip install tensorflow_decision_forests
# TF-DF requires Tensorflow < 2.15 or tf_keras
!pip install tf_keras

In [ ]:
import os
# Keep using Keras 2
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import tensorflow_decision_forests as tfdf

import numpy as np
import pandas as pd
import tensorflow as tf
import tf_keras
import math
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ndcg_score
from sklearn.metrics import average_precision_score

In [ ]:
# Сюда будем собирать результаты
df_res = pd.DataFrame(columns=['algo', 'score'])

## Для начала попробуем XGBRanker

In [ ]:
df = pd.read_csv('/content/train_df.csv')
df_test = pd.read_csv('/content/test_df.csv')

In [ ]:
df['target'].value_counts()

0    14759
1      322
Name: target, dtype: int64

### Удалим одинаковые строки и столбцы, у которых 1 значение в каждой строке

In [ ]:
df.drop_duplicates(inplace=True)

drop_col = []
for col in df.columns:
    if col == 'target':
        continue
    if len(df_test[col].value_counts()) <= 2:
        drop_col.append(col)
        df = df.drop(col, axis=1)

In [ ]:
df = df.sort_values('search_id')

In [ ]:
X_train, y_train, qid_train = df.iloc[:, 1:-1], df['target'], df['search_id']

In [ ]:
parameters = {
    'lambdarank_pair_method': 'mean',
    'lambdarank_unbiased': False,
    'learning_rate': 0.01,
    'max_depth': 2,
    'n_estimators': 10,
    'ndcg_exp_gain': False,
    'objective': 'rank:ndcg',
}

In [ ]:
model = xgb.XGBRanker(**parameters)
model.fit(X_train, y_train, qid=qid_train)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          lambdarank_pair_method='mean', lambdarank_unbiased=False,
          learning_rate=0.01, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=2,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=10,
          n_jobs=None, ...)

In [ ]:
df_test.rename(columns={'search_id' : 'qid'}, inplace=True)

In [ ]:
df_test.sort_values('qid', inplace=True)

In [ ]:
df_test.drop(drop_col, axis=1, inplace=True)

In [ ]:
model.score(df_test.drop('target', axis=1), df_test.target)

0.9192847645054288

In [ ]:
qid_mass = pd.unique(df_test['qid'])

In [ ]:
def ndcg(y_score, y_true, k):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)

In [ ]:
ndcg_ = list()

for i, qid in enumerate(qid_mass):
    y = df_test[df_test['qid'] == qid]['target']
    X_valid = df_test[df_test['qid'] == qid].iloc[:, 1:-1]
    if np.sum(y) == 0:
        continue

    p = model.predict(X_valid)

    idcg = ndcg(y, y, k=10)
    ndcg_.append(ndcg(p, y, k=100) / idcg)

In [ ]:
np.mean(ndcg_)

0.6770548299528976

In [ ]:
df_res.loc[len(df_res)] = ('XGBRanker', np.mean(ndcg_))

In [ ]:
df_res

,algo,score
0,XGBRanker,0.677055


## Попробуем использовать keras

In [ ]:
df = pd.read_csv('/content/train_df.csv')
df_test = pd.read_csv('/content/test_df.csv')

## Также чистим данные от неинформативных столбцов и строк

In [ ]:
df.drop_duplicates(inplace=True)

drop_col = []
for col in df.columns:
    if col == 'target':
        continue
    if len(df_test[col].value_counts()) <= 2:
        drop_col.append(col)
        df = df.drop(col, axis=1)

In [ ]:
df['search_id'] = df['search_id'].apply(lambda x: str(x))
df_test['search_id'] = df_test['search_id'].apply(lambda x: str(x))


In [ ]:
dataset_ds = tfdf.keras.pd_dataframe_to_tf_dataset(df, label="target", task=tfdf.keras.Task.RANKING)

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    task=tfdf.keras.Task.RANKING,
    ranking_group="search_id",
    num_trees=50)

model.fit(dataset_ds)

Use /tmp/tmp5bsh4g6e as temporary training directory
Reading training dataset...
Training dataset read in 0:00:13.870298. Found 15069 examples.
Training model...
Model trained in 0:00:16.608730
Compiling model...
Model compiled.


In [ ]:
def ndcg(y_score, y_true, k):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)

In [ ]:
qiud = pd.unique(df_test['search_id'])

In [ ]:
ndcg_ = list()

for q in qiud:
    y_q_test = df_test[df_test['search_id'] == q]['target']
    serving_dataset_df = df_test[df_test['search_id'] == q]
    # Remove the columns for group and relevance, not needed for predictions.
    serving_dataset_df = serving_dataset_df.drop(['target', 'search_id'], axis=1)
    # Convert to a Tensorflow dataset
    serving_dataset_ds = tfdf.keras.pd_dataframe_to_tf_dataset(serving_dataset_df, task=tfdf.keras.Task.RANKING)

    y_pred = model.predict(serving_dataset_ds, verbose=False)

    if np.sum(y_q_test) == 0:
        continue

    tmp = []
    for el in y_pred:
      tmp.append(float(el))
    y_pred = tmp

    y_q_test = [list(y_q_test)]

    idcg = ndcg(y_q_test, y_q_test, k=10)
    ndcg_.append(ndcg(y_pred, y_q_test, k=10) / idcg)




<ipython-input-28-19a58a10138f>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tmp.append(float(el))
<ipython-input-28-19a58a10138f>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tmp.append(float(el))
<ipython-input-28-19a58a10138f>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tmp.append(float(el))
<ipython-input-28-19a58a10138f>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you 

In [ ]:
np.mean(ndcg_)

0.5680858580458242

In [ ]:
df_res.loc[len(df_res)] = ('keras.GradientBoostedTreesModel', np.mean(ndcg_))

In [ ]:
df_res

,algo,score
0,XGBRanker,0.677055
1,keras.GradientBoostedTreesModel,0.568086


## Попробуем CatBoost

In [ ]:
! pip install catboost

In [ ]:
from catboost import CatBoostRanker, Pool, MetricVisualizer

In [ ]:
df = pd.read_csv('/content/train_df.csv')
df_test = pd.read_csv('/content/test_df.csv')

In [ ]:
# Чистим данные
df.drop_duplicates(inplace=True)

drop_col = []
for col in df.columns:
    if col == 'target':
        continue
    if len(df_test[col].value_counts()) <= 1:
        drop_col.append(col)
        df = df.drop(col, axis=1)

In [ ]:
feature_cols = df.iloc[:, 1:-1].columns
target = 'target'


In [ ]:
train = Pool(
    data = df[feature_cols],
    label = df[target],
    group_id = df['search_id']
)

In [ ]:
parameters = {
    'iterations': 2,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10'],
    'random_seed': 0,
}

ranker = CatBoostRanker(**parameters)

In [ ]:
ranker.fit(train, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	total: 238ms	remaining: 238ms
1:	total: 431ms	remaining: 0us


In [ ]:
df_test.rename(columns={'search_id' : 'qid'}, inplace=True)

In [ ]:
df_test.sort_values('qid', inplace=True)

In [ ]:
df_test.drop(drop_col, axis=1, inplace=True)

In [ ]:
ranker.score(df_test.drop(['target', 'qid'], axis=1), df_test.target, group_id=df_test['qid'])

0.8693271799107375

In [ ]:
qid_mass = pd.unique(df_test['qid'])

In [ ]:
def ndcg(y_score, y_true, k):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


In [ ]:
qids = pd.unique(df_test['qid'])

In [ ]:
ndcg_ = list()

for i, qid in enumerate(qids):
    X_query = df_test[df_test['qid'] == qid].drop(['qid', 'target'], axis=1)
    y = df_test[df_test['qid'] == qid]['target']

    if np.sum(y) == 0:
        continue

    p = ranker.predict(X_query)

    idcg = ndcg(y, y, k=10)
    ndcg_.append(ndcg(p, y, k=100) / idcg)

In [ ]:
np.mean(ndcg_)

0.5845200529629752

In [ ]:
df_res.loc[len(df_res)] = ('CatBoost', np.mean(ndcg_))

In [ ]:
df_res

,algo,score
0,XGBRanker,0.677055
1,keras.GradientBoostedTreesModel,0.568086
2,CatBoost,0.584520


## Попробуем LightGBM

In [ ]:
! pip install lightgbm

In [ ]:
df = pd.read_csv('/content/train_df.csv')
df_test = pd.read_csv('/content/test_df.csv')

In [ ]:
df.sort_values('search_id', inplace=True)
df_test.sort_values('search_id', inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)

drop_col = []
for col in df.columns:
    if col == 'target':
        continue
    if len(df_test[col].value_counts()) <= 1:
        drop_col.append(col)
        df = df.drop(col, axis=1)

In [ ]:
df_test.drop(drop_col, axis=1, inplace=True)

# стандартизация улучшила метрику
scaler = StandardScaler()
df.iloc[:, 1:-1] = scaler.fit_transform(df.iloc[:, 1:-1])

In [ ]:
X_train, y_train, qid_train = df.iloc[:, 1:-1], df.target, df['search_id']
X_test, y_test, qid_test = df_test.iloc[:, 1:-1], df_test.target, df_test['search_id']

In [ ]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
_, group_train = np.unique(qid_train, return_counts=True)

In [ ]:
from lightgbm import LGBMRanker

gbm = LGBMRanker()
gbm.fit(X_train, y_train, group=group_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007341 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12495
[LightGBM] [Info] Number of data points in the train set: 15069, number of used features: 75


LGBMRanker()

In [ ]:
predictions = []

for group in np.unique(qid_test):
    X_test = df_test[df_test['search_id'] == group].iloc[:, 1:-1]
    preds = gbm.predict(X_test)
    predictions.extend(preds)

In [ ]:
def ndcg(y_score, y_true, k):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)

In [ ]:
qids = np.unique(qid_test)

In [ ]:
ndcg_ = list()

for i, qid in enumerate(qids):
    y = y_test[qid_test == qid]

    if np.sum(y) == 0:
        continue
    X_test = df_test[df_test['search_id'] == qid].iloc[:, 1:-1]
    p = gbm.predict(X_test)

    idcg = ndcg(y, y, k=10)
    ndcg_.append(ndcg(p, y, k=100) / idcg)

In [ ]:
np.mean(ndcg_)

0.6613790452404316

In [ ]:
df_res.loc[len(df_res)] = ('LightGBM', np.mean(ndcg_))

In [ ]:
df_res

,algo,score
0,XGBRanker,0.677055
1,keras.GradientBoostedTreesModel,0.568086
2,CatBoost,0.584520
3,LightGBM,0.661379
